# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv("/datasets/data.csv")
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
df.info()
#Таблица имеет 21525 строк и 12 столбцов.
#Видим одинаковое количество пропущенных данных в столбцах days_employed и total_income.
#Причиной этого может быть, к примеру, потеря части данных, когда несколько баз данных объединяли в одну.
#Или же по части клиентов намеренно скрыли информацию по их доходам и опыту работы.
#display(df[df['days_employed'].isna()].head(20)) #при желании можно вывести строки с пропущенными данными

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Для оценки категориальных переменных (education, family_status, gender, income_type, purpose) воспользуемся методом value_counts (), а для количественных переменных (children, days_employed, dob_years, total_income) методом unique () и/или min()/max().** 

In [3]:
df['education'].value_counts()
#В столбце "education" попадаются поля, заполненнные полностью или частично прописными буквами.
#Судя по вариантам написания, можно предположить, что массив данных был получен в результате объединения 3 разных баз.

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [4]:
df['family_status'].value_counts()
#Видим прописную букву в статусе "Не женат / не замужем", но в данном случае она не помешает анализу данных, поскольку применена для всех подобных статусов.

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [5]:
df['gender'].value_counts()
#По одному из клиентов в столбце пол присвоен признак "XNA".
#Если мы захотим выяснить, кто реже допускает просрочки: мужчины или женщины, эту строку можно будет без ущерба игнорировать

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [6]:
df['income_type'].value_counts()
#Ожидаемо, что большинство из выборки работают по найму. Ошибок в данных нет, хотя безработный, студент и женщина/мужчина в декрете смотрятся странно в числе людей, которые получили кредит в банке на столь существенные цели (см. далее)

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [7]:
df['purpose'].value_counts()
#Как видим, в основном кредиты берут на сделки с недвижимостью.
#Эти данные в дальнейшем нужно будет категоризировать для удобства анализа.

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [8]:
print(sorted(df['children'].unique(),reverse=False))
#Вероятнее всего, мы наблюдаем ошибки в данных по количеству детей (-1 ребенок и 20 детей). 
df['children'].value_counts()
#Теоретически, возможно иметь 20 детей в одной семье, но слишком большой разрыв между последующим значением (5)
#по величине значением, а также общее количество таких семей в количестве (76 семей, что больше чем семей с 4 детьми). 
#Это нам говорит скорее об ошибке при вводе данных. 
#Аналогично, -1 ребенок в 47 семьях говорит нам об ошибке при вводе данных.

[-1, 0, 1, 2, 3, 4, 5, 20]


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [9]:
#df['days_employed'].max() #unique()
print("{:,}".format(df['days_employed'].min()).replace(',', ' '))
print("{:,}".format(df['days_employed'].max()).replace(',', ' '))
#Ого, мы нашли долгожителя и ветерана труда, который отработал 401 755 дней, то есть 1 100 лет.
display(df.loc[df["days_employed"] == df['days_employed'].max()])
#Если посмотреть конкретно этого заемщика, то ему 56 лет.
#Столбец "days_employed" требует особого внимания. Возможно, часть данных указана не в днях, а в часах.
#Для максимального значения 401 755 часов это составит 45,8 лет, что при возрасте заемщика 56 лет 
#выглядит правдоподобным, если заемщик работал по совместительтву какой-то период времени.
print("{:,}".format(df['days_employed'].median()).replace(',', ' '))
#Данные в столбце "days_employed" указаны в основном со знаком минус (об этом нам говорит отрицательная медиана) и при этом имеет формат дробного числа, хотя должен отражать количество дней общего стажа.

-18 388.949900568383
401 755.40047533


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью


-1 203.369528770489


In [10]:
print(sorted(df['dob_years'].unique()))
#видим заемщиков, по которым указан возраст 0

print(df[df['dob_years'] == 0]['dob_years'].count())
#находим, что таких 101 человек

[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
101


In [11]:
print("{:,.2f}".format(df['total_income'].min()).replace(',', ' '))
print("{:,.2f}".format(df['total_income'].max()).replace(',', ' '))
#максимальное значение в столбце доходы - заемщик с доходом 2,2 млн.руб. в месяц, что более чем в 100 раз выше миниального дохода.
display(df.loc[df["total_income"] == df['total_income'].max()])
#если рассмотреть конкретно этого заемщика, то видим, что это мужчина 44 лет, который владеет бизнесом, поэтому такой доход вполне реален.
#однако при заполнении пропущенных значений лучше использовать медиану, иначе средняя может быть сильно завышена.

20 667.26
2 265 604.03


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью


#### **Вывод**

В процессе анализа данных были выявлены следующие проблемы, мешающие обработке данных:
- данные в столбце "days_employed" указаны в основном со знаком минус и при этом в виде дробного числа; есть гипотеза, что частично информация указана не в днях,а в часах, поскольку в противном случае имеется заемщик, который при возрасте 56 лет отработал 1100 лет;
- имеются пропущенные данные в столбцах days_employed и total_income;
- по 101 заемщику указан возраст 0 лет;
- в столбце "education" попадаются поля, заполненнные полностью или частично прописными буквами - это может скрывать наличие неявных дубликатов;
- по одному из клиентов отсутсвуют данные в части его пола;
- вероятнее всего, мы наблюдаем ошибки в данных в столбце "children": -1 ребенок в 47 семей и 20 детей сразу в 76 семьях).

## Шаг 2. Предобработка данных

### Обработка пропусков

В days_employed и total_income имеются пропущенные данные. Заполним их медианными значениями по возрастным группам. Однако в столбце dob_years у 101 человека возраст равен 0. Поэтому сначала  нужно обработать столбец с данными по возрасту.

In [12]:
#Поскольку в таблице возраст нет выдающихся значений, поэтому отсутствующие значения можно заменить
#средним арифметическим. Для этого введем переменную age_mean и воспользуемся методом mean() для всех ненулевых значений.
display(df[df['dob_years'] != 0]['dob_years'].mean())
age_mean = 43

43.497479462285284

In [13]:
#Теперь заменим нулевые значения на средний арфиметический возраст.
df['dob_years'] = df['dob_years'].replace(0, age_mean)

In [14]:
print(df['dob_years'].unique())
#Нулевые значения в столбце возраст исчезли.

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]


In [15]:
#В столбце days_employed мы видим как отрицательные, так и положительные значения. 
#Вычислим модуль по этому столбцу и выведем новое минимальное значение.
df['days_employed'] = abs(df['days_employed'])
print("{:,.2f}".format(df['days_employed'].min()).replace(',', ' '))
#Мы видим, что минимальное значение по столбцу days_employed теперь 24.
#Это означает,что значения в этом столбце все же указаны не в часах, по крайней мере не все.

24.14


In [16]:
#Отсортируем значения столбца days_employed по убыванию и более внимательно посмотрим на данные.
#Сначала идут значение больше 400 000, а затем резкий скачок до 18 388. 
#print(sorted(df['days_employed'].unique(),reverse=True))
#Поскольку максимальный возраст заемщика 75 лет, то при допущении, что он начал работать в 16 лет и продолжает работать до сих пор, его трудовой стаж составит 59 лет, или 21 535 дней.
#Таким образом, все значения выше являются аномальными. Посчитаем их общее количество.
df[df['days_employed'] > (75-16)*365]['days_employed'].count()
#Таких строк оказалось 3445, что составляет 16% от массива. Эти строки нельзя удалять, поскольку это может серьезно исказить результат исследования.

3445

In [17]:
#Переведем значения выше 21 535 из часов в дни.
df['days_employed'] = df['days_employed'].apply(lambda i: i/24 if i > 21535 else i)   
print(df['days_employed'].max())
#Значения выше 21000 исчезли.

18388.949900568383


In [18]:
#Теперь заменим пропущенные значения в столбце days_employed в разрезе возрастных групп 
#Для этого создадим отдельный столбец с возрастными категориями "менее 30 лет", "30 - 49 лет", "50 - 64 лет", "пенсионеры"
def age_group(age):
    if age < 30:
        return 'менее 30 лет'
    if age >= 30 and age < 50:
        return 'от 30 до 49 лет'
    if age >= 50 and age < 65:
        return 'от 50 до 65 лет'
    return 'пенсионеры' 
df['age_group'] = df['dob_years'].apply(age_group)
df['age_group'].value_counts()

от 30 до 49 лет    11146
от 50 до 65 лет     6297
менее 30 лет        3183
пенсионеры           899
Name: age_group, dtype: int64

In [19]:
#определим медианный стаж работы для каждой возрастной группы
days_employed_median = df.groupby('age_group')['days_employed'].median()
display(days_employed_median)

age_group
менее 30 лет         999.028882
от 30 до 49 лет     1809.894126
от 50 до 65 лет     9151.490125
пенсионеры         15020.871042
Name: days_employed, dtype: float64

In [20]:
#сохраним медианные значения для каждой возрастной группы в виде переменной
days_employed_less_30 = 1243
days_employed_less_50 = 2590
days_employed_less_65 = 8743
days_employed_more_65 = 13636

In [21]:
#теперь заполним пропуски в столбце days_employed в разрезе возрастных групп
def fillna_days_employed(row):
    days_employed = row['days_employed']
    age_group = row['age_group']
    if age_group == 'менее 30 лет':
        return days_employed_less_30
    if age_group == 'от 30 до 49 лет':
        return days_employed_less_50
    if age_group == 'от 50 до 65 лет':
        return days_employed_less_65
    return days_employed_more_65 

df.loc[df.days_employed.isna(), "days_employed"] = df[df.days_employed.isna()].apply(fillna_days_employed, axis=1)


In [22]:
#проверим, что опыт работы не превышает возраст заемщика
def age_check(row):
    if row['days_employed']/365 < row['dob_years']:
        return 'ok'
    return 'ошибка' 

df['age_check'] = df.apply(age_check, axis=1)
df['age_check'].value_counts()

#Обнаружилось 58 строк, где опыт работы превышает возраст заемщика.
#display(df[df['age_check'] == "ошибка"].sort_values(by='days_employed',ascending=False).head(10)) - при желании можно вывести список из таких заемщиков
#Обратим внимание, что у практически у всех стоит тип занятости "пенсионер", но при этом людям по 30-40 лет. 
#Можно предположить, что это бывшие военные, у которых в некоторых случаях возможен повышенный учет стажа - например, при службе на флоте или в особых условиях месяц службы будет приравнен к 2 месяцам трудовой деятельности.

ok        21467
ошибка       58
Name: age_check, dtype: int64

In [23]:
#В столбце "children" мы наблюдаем ошибки в данных: -1 ребенок в 47 семей и 20 детей в 76 семьях.
#Можно предположить, что раз эти значения отличны от нуля, то в семьях заемщиков имеются дети, а ошибки появились в процессе внесения информации.
#Устраним данные ошибки, заменив -1 на 1, а 20 на 2
df['children'] = abs(df['children'])
df['children'] = df['children'].replace(20, 2)
df['children'].value_counts()
#теперь данные однородны

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [24]:
#Поскольку в выборке есть заемщики с доходом свыше 2 млн.руб., определим медианный доход для каждой возрастной группы
total_income_median = df.groupby('age_group')['total_income'].median()
display(total_income_median)

age_group
менее 30 лет       142141.582776
от 30 до 49 лет    154493.872983
от 50 до 65 лет    136439.063467
пенсионеры         115446.195141
Name: total_income, dtype: float64

In [25]:
#сохраним медианные значения по доходу для каждой возрастной группы в виде переменной
total_income_less_30 = 142142
total_income_less_50 = 154494
total_income_less_65 = 136439
total_income_more_65 = 115446

In [26]:
#теперь заполним пропуски в столбце days_employed в разрезе возрастных групп
def fillna_total_income(row):
    total_income = row['total_income']
    age_group = row['age_group']
    if age_group == 'менее 30 лет':
        return total_income_less_30
    if age_group == 'от 30 до 49 лет':
        return total_income_less_50
    if age_group == 'от 50 до 65 лет':
        return total_income_less_65
    return total_income_more_65 

df.loc[df.total_income.isna(), "total_income"] = df[df.total_income.isna()].apply(fillna_total_income, axis=1)
display(df['total_income'].isna().sum())
#все пропущенные значения в столбце total_income заполнены

0

In [27]:
#финальная проверка на наличие пропусков
df.info()
#теперь во всех столбцах количество строк одинаковое - пропусков больше нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
age_check           21525 non-null object
dtypes: float64(2), int64(5), object(7)
memory usage: 2.3+ MB


**Вывод**
Процесс обработки пропусков оказался непростым, поскольку в столбце days_employed большая часть данных была отрицательной, а другая в разы отличалась от модуля медианы. Столбец total_income также потребовал взвешенного подхода. В итоге пропущенные данные заполнялись медианными значениями для каждой возрастной группы.

### Замена типа данных

In [28]:
#Мы обнаружили, что в столбцах days_employed и total_income данные имеют дробный тип float.
#Переведем их в целые числа, используя метод astype()
df['days_employed'] = df['days_employed'].astype("int")
df['total_income'] = df['total_income'].astype("int")
display(df['days_employed'].dtypes)
display(df['total_income'].dtypes)

dtype('int64')

dtype('int64')

**Вывод**
Для удобства последующей работы мы изменили тип данных в столбцах days_employed и total_income с вещественного на целое число.

### Обработка дубликатов

In [29]:
#Проверим наличие явных дубликатов
df.duplicated().sum()
#Первичный анализ выявил 54 дубликата, но их может быть и больше, когда мы приведем прописные буквы к нижнему регистру

54

In [30]:
#Приведем все прописные буквы в столбце "education" к нижнему регистру
df['education'] = df['education'].str.lower()
df['education'].value_counts()
#Осталось только 5 вариантов образования

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [31]:
#Проверим, изменилось ли количество дубликатов
df.duplicated().sum()
#После данной операции число дубликатов возросло с 54 до 71

71

In [32]:
#Теперь удалим дубликаты и перепишем таблицу с новой индексацией.
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()
#После обработки дубликаты исчезли.

0

**Вывод**
Предварительный анализ выявил 54 дубликата. С помощью метода value_counts() мы определили возможное наличие неявных дубликатов в столбце 'education'. После приведения всех букв в нижний регистр число дубликатов возросло до 71.

### Лемматизация

In [33]:
#Проведем лематизацию столбца 'purpose' для последующей категоризации данных 
#Для этого сначала объединим все данные столбца purpose в одну строку, затем лемматизируем ее и посчитаем число упоминаний. 

from pymystem3 import Mystem
m = Mystem()
from collections import Counter
purposes  = " ".join(df['purpose'])
lemmas = m.lemmatize(purposes)
print(Counter(lemmas))

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


In [34]:
#Составим словарь унифицированных целей кредита
purpose_list = ["недвижимость", "жилье", "автомобиль", "образование", "свадьба", "строительство", "ремонт"]
#Слово "операция" в данном случае не имеет значения как расходы на лечение, поэтому не учитывается.
#При этом понятия "жилье" и "недвижимость" в данном случае синонимичны, так что их все можно объединить в категорию "недвижимость"

**Вывод**
На основании статистики в основном люди берут кредит на сделку с недвижимостью, покупку автомобиля, получение образования и проведение свадьбы.

### Категоризация данных

In [35]:
#Добавим новый столбец с унифицированной целью кредита. 
#При этом в категорию недвижимость также будет включена лемма "жилье"
def purpose_group(purpose):
    if "свадьб" in purpose:
        return "свадьба"
    if "образован" in purpose:
        return "образование"
    if "автомоб" in purpose:
        return "автомобиль"
    if ("недвиж" in purpose) or ("жил" in purpose):
        return "недвижимость"
    return "прочие"

df['purpose_group'] = df['purpose'].apply(purpose_group)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,age_check,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 30 до 49 лет,ok,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 30 до 49 лет,ok,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 30 до 49 лет,ok,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,от 30 до 49 лет,ok,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,от 50 до 65 лет,ok,свадьба


In [36]:
#Проверим, не остались ли строки, которые не покрываются данными категориями (для этого мы ввели категорию "прочие"), а также не появились ли новые дубликаты
display(df['purpose_group'].value_counts())
#Весь массив теперь разбит на 4 цели получения кредита, новых дубликатов не обнаружено
display(df.duplicated().sum())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

0

In [37]:
#Для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок" необходимо разбить заемщиков на подгруппы по уровню доходов
#Создадим отдельный столбец с разбивкой по уровню дохода "менее 100 тыс. руб.", "100 - 199 тыс.руб.", "200 - 499 тыс.руб.", "свыше 500 тыс.руб."
def income_group(income):
    if income < 100000:
        return 'менее 100 тыс. руб.'
    if income >= 100000 and income < 200000:
        return '100 - 199 тыс.руб.'
    if income >= 200000 and income < 500000:
        return '200 - 499 тыс.руб.'
    
    return "свыше 500 тыс.руб." 
df['income_group'] = df['total_income'].apply(income_group)
df['income_group'].value_counts()

100 - 199 тыс.руб.     11925
200 - 499 тыс.руб.      4844
менее 100 тыс. руб.     4463
свыше 500 тыс.руб.       222
Name: income_group, dtype: int64

**Вывод**
С помощью лемматизации мы смогли выделить ключевые цели получения кредита и провести категоризацию для дальнешего решения задачи. Теперь весь массив разбит на 4 цели получения кредита. Также мы разбили заемщиков на 4 категории по уровню получаемого дохода для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?".

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
#Для решения этой задачи выведем % просрочек по кредиту внутри групп "бездетные" и "с детьми" 
#как отношение количества строк с просрочками к общему количеству строк внутри выборки.
families_childless = df[df['children'] == 0]['children'].count()
families_with_children = df[df['children'] != 0]['children'].count()
families_childless_debt = len(df[(df['children'] == 0) & (df['debt'] == 1)])
families_with_children_debt = len(df[(df['children'] != 0) & (df['debt'] == 1)])
families_childless_ratio = families_childless_debt/families_childless
families_with_children_ratio = families_with_children_debt/families_with_children
print("Доля просрочек в семьях без детей: {:.1%}".format(families_childless_ratio))
print("Доля просрочек в семьях с детьми: {:.1%}".format(families_with_children_ratio))

Доля просрочек в семьях без детей: 7.5%
Доля просрочек в семьях с детьми: 9.2%


In [39]:
#А теперь создадим сводную таблицу в разрезе столбцов 'children' и 'debt', 
#чтобы посмотреть, есть ли зависимость между просрочкой и количеством детей
data_pivot = df.pivot_table(index='children', columns='debt', values='gender', aggfunc='count')
data_pivot['total'] = data_pivot[0] + data_pivot[1]
data_pivot['ratio'] = data_pivot[1] / data_pivot['total']
display(data_pivot.sort_values(by='ratio',ascending=False).style.format({'ratio':'{:.1%}'.format}))

debt,0,1,total,ratio
children,,,,
4,37,4,41,9.8%
2,1926,202,2128,9.5%
1,4410,445,4855,9.2%
3,303,27,330,8.2%
0,13028,1063,14091,7.5%
5,9,nan,nan,nan%


**Вывод**
Анализ показал, что в семьях с детьми доля просрочки в среднем на 1,7% выше, чем в семьях без детей.
Если же смотреть просрочки внутри семей с детьми, то здесь нет четкой зависимости, что с большим количеством детей платежная дицсиплина падает или растет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
#Создадим сводную таблицу в разрезе столбцов 'family_status' и 'debt'. 
data_pivot_family_status = df.pivot_table(index='family_status', columns='debt', values='gender', aggfunc='count')
data_pivot_family_status['total'] = data_pivot_family_status[0] + data_pivot_family_status[1]
data_pivot_family_status['ratio'] = data_pivot_family_status[1] / data_pivot_family_status['total']
display(data_pivot_family_status.sort_values(by='ratio',ascending=False).style.format({'ratio':'{:.1%}'.format}))

debt,0,1,total,ratio
family_status,,,,
Не женат / не замужем,2536,274,2810,9.8%
гражданский брак,3763,388,4151,9.3%
женат / замужем,11408,931,12339,7.5%
в разводе,1110,85,1195,7.1%
вдовец / вдова,896,63,959,6.6%


**Вывод**
На основании полученных данных, можно сделать вывод, что наименее дисциплинированны холостяки/холостячки, а также те, кто живут в гражданском браке - их доля просрочки 9,3%-9,8%, в то время как у семейных, а также те, кто ранее был женат/замужем доля просрочки 6,6% - 7,5%. Отдельно стоит отметить минимальную долю просрочек среди тех, кто потерял супруга/супругу - 6,6%

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
#Создадим сводную таблицу в разрезе столбцов 'income_group' и 'debt'. 
data_income_group = df.pivot_table(index='income_group', columns='debt', values='gender', aggfunc='count')
data_income_group['total'] = data_income_group[0] + data_income_group[1]
data_income_group['ratio'] = data_income_group[1] / data_income_group['total']
display(data_income_group.sort_values(by='ratio',ascending=False).style.format({'ratio':'{:.1%}'.format}))

debt,0,1,total,ratio
income_group,,,,
100 - 199 тыс.руб.,10896,1029,11925,8.6%
менее 100 тыс. руб.,4109,354,4463,7.9%
200 - 499 тыс.руб.,4500,344,4844,7.1%
свыше 500 тыс.руб.,208,14,222,6.3%


**Вывод**
Однозначной зависимости между уровнем дохода и возвратом кредита в срок не наблюдается: у заемщиков с доходом менее 100 тыс. руб. доля просрочек ниже, чем у тех, кто имеет доход 100 - 199 тыс.руб. При этом состоятельные клиенты с доходом свыше 500 тыс.руб в месяц имеют минимальный уровень просрочки в 6,3%

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
#Создадим сводную таблицу в разрезе столбцов 'purpose_group' и 'debt'. 
data_pivot_purpose = df.pivot_table(index='purpose_group', columns='debt', values='gender', aggfunc='count')
data_pivot_purpose['total'] = data_pivot_purpose[0] + data_pivot_purpose[1]
data_pivot_purpose['ratio'] = data_pivot_purpose[1] / data_pivot_purpose['total']
display(data_pivot_purpose.sort_values(by='ratio',ascending=False).style.format({'ratio':'{:.1%}'.format}))

debt,0,1,total,ratio
purpose_group,,,,
автомобиль,3903,403,4306,9.4%
образование,3643,370,4013,9.2%
свадьба,2138,186,2324,8.0%
недвижимость,10029,782,10811,7.2%


**Вывод**
Из сводной таблицы сразу видно, что доля просрочек в кредитах на покупку автомобиля и получение образования самая высокая - более 9%, в то время как кредиты на недвижимость показывают минимальное значение 7,2%. Кредиты на свадьбу занимают промежуточную позицию с долей просрочек 8%.

In [43]:
#Дополнительно проверим влияние образования на финансовую дисциплину
data_pivot_education = df.pivot_table(index='education', columns='debt', values='gender', aggfunc='count')
data_pivot_education['total'] = data_pivot_education[0] + data_pivot_education[1]
data_pivot_education['ratio'] = data_pivot_education[1] / data_pivot_education['total']
display(data_pivot_education.sort_values(by='ratio',ascending=False).style.format({'ratio':'{:.1%}'.format}))

debt,0,1,total,ratio
education,,,,
начальное,251,31,282,11.0%
неоконченное высшее,676,68,744,9.1%
среднее,13808,1364,15172,9.0%
высшее,4972,278,5250,5.3%
ученая степень,6,nan,nan,nan%


**Вывод**
мы видим, что доля просрочек у тех, кто получил высшее образование, почти в 2 раза ниже (порядка 5%), чем у тех, кто либо еще находится в процессе его получения, либо ограничился средним образованием (примерно 9%).

In [44]:
#Дополнительно проверим влияние пола на финансовую дисциплину
data_pivot_gender = df.pivot_table(index='gender', columns='debt', values='income_type', aggfunc='count')
data_pivot_gender['total'] = data_pivot_gender[0] + data_pivot_gender[1]
data_pivot_gender['ratio'] = data_pivot_gender[1] / data_pivot_gender['total']
display(data_pivot_gender.sort_values(by='ratio',ascending=False).style.format({'ratio':'{:.1%}'.format}))

debt,0,1,total,ratio
gender,,,,
M,6532,747,7279,10.3%
F,13180,994,14174,7.0%
XNA,1,nan,nan,nan%


**Вывод**
Несмотря на то что среди заемщиков женщин почти в два раза больше, чем мужчин, доля просрочек среди них на 3% ниже.

## Шаг 4. Общий вывод

**По итогам анализа можно сделать следующие выводы влияния того или иного фактора на погашения кредита в срок:**
- холостые/незамужние, а также лица, проживающие в гражданском браке больше склонны допускать просрочки по кредитам;
- наличие детей в семье увеличвает риск возникновения просрочки;
- уровень дохода напрямую не влияет на уровень возврата кредита в срок;
- наиболее рискованые кредиты выдаются на покупку автомобиля или получение образования;
- мужчины менее организованы и чаще допускают просрочки по кредитам;
- люди без высшего образования гораздо менее дисциплинированы в части выплат по кредитам.